In [1]:
#coding=utf-8
import time
import re
import os
import requests
import re
from bs4 import BeautifulSoup
from multiprocessing import Pool

In [2]:
# 初始化一些值
headers = {
    'Authorization': '',
    'Host': 'sspai.com',
    'Referer': 'https://sspai.com/',
    'User-Agent': 'Mozilla/5.0',
   'Cookie': ''
}
total = 0  # 文章总数
offset = 0 # 偏移量
limit = 50 # 每次取 50 条

In [42]:
# 时间戳转成看得懂的日期时间
def get_time(timestamp):
    time_local = time.localtime(timestamp)
    dt = time.strftime("%Y-%m-%d %H:%M:%S",time_local)    
    return dt

In [43]:
def get_list():
    global total,offset,limit
    article_dict = {}
    while True:
        url = "https://sspai.com/api/v1/articles?offset={0}&limit={1}".format(str(offset),str(limit))
        print('fetching',url)
        try:
            resp = requests.get(url,headers=headers)
            j = resp.json()
            data = j['list']
            total = j['total']
        except:
            print('get list failed')
        for article in data:
            aid = article['id']
            akeys = article_dict.keys()
            print(aid)
            if aid not in akeys:
                article_dict[aid] = [article['title'],get_time(article['released_at'])]
    
        if (offset+limit) >= total:
            break
            
        offset += 50
            
        time.sleep(3)
    
    with open('sspai_ids.txt', 'w',encoding='utf-8') as f:
        items = sorted(article_dict.items())
        for item in items:
            f.write('{0} {1} {2}\n'.format(item[0],item[1][0],item[1][1]))

In [3]:
def get_html(url):
    title = re.sub('[\/:*?"<>|]', '-', url[1])  # 正则过滤非法文件字符
    file_name = '%03d. %s.html' % (url[2], title)
    if os.path.exists(file_name):
        print(title, 'already exists.')
        return
    else:
        print('saving', title)
    html = requests.get("https://sspai.com/post/"+url[0], headers=headers).text
    soup = BeautifulSoup(html, 'lxml')
    try:
        content = soup.find("div", {"class": "article-detail"})
        content = content.article.prettify()
    except:
        print("saving", title, "error")
        return
    content = content.replace('h1>', 'h2>')
    content = '<!DOCTYPE html><html><head><meta charset="utf-8"></head><body><h1>%s</h1>%s</body></html>' % (
        title, content)
    with open(file_name, 'w', encoding='utf-8') as f:
        f.write(content)
    time.sleep(3)

def get_details():
    detailList = []
    with open('sspai_ids.txt', 'r', encoding='utf-8') as f:
        i = 1
        for line in f:
            lst = line.strip().split(' ')
            url = lst[0]
            title = '_'.join(lst[1:])
            detailList.append((url,title,i))
            i += 1
            if i >= 10:
                break
    return detailList

In [ ]:
if __name__ == '__main__':
#     get_list()
    pool = Pool(processes=10) # 进程池
    s1 = time.time()
    pool.map(get_html,get_details())
    pool.close()
    pool.join()
    print("time used ",int(time.time()-s1))

In [6]:
resList[0]

{'Tag_id': 0,
 'advertisement': None,
 'advertisement_id': 0,
 'advertisement_url': '',
 'all_comments_count': 0,
 'allow_comment': True,
 'applications': None,
 'author': {'alipay': '',
  'articles_count': 0,
  'articles_word_count': 0,
  'avatar': '2017/09/17/0e4fbebd5468b85aa6be5d7c42221f51.jpg',
  'avatar_id': 413907,
  'banned_until': 0,
  'bio': '自由职业者，一名程序员，平时会写一些稿子。',
  'blocked': False,
  'city': '',
  'comment_replys_count': 0,
  'comments_count': 0,
  'country': '',
  'email': None,
  'email_matrixs': True,
  'email_messages': True,
  'followed': False,
  'followed_count': 0,
  'following_count': 0,
  'franchised_writer': False,
  'id': 769301,
  'id_card_attachment_ids': None,
  'id_card_attachments': None,
  'is_email_verified': False,
  'is_series_author': False,
  'liked_count': 5,
  'message_on': False,
  'money': 0,
  'national_area_id': 0,
  'nickname': '西秦公子',
  'no_read': False,
  'occupation': '自由职业者。',
  'phone': None,
  'professional_writer': False,
  'province':

In [7]:
print(j["total"])

14883
